In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig

import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import utils_tSNE as utsne

from matplotlib.backends.backend_pdf import PdfPages
import utils_preproc as upre

#mpl.use('Agg')

############################

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'


rawname_= 'S01_off_hold'

In [ ]:
src_fname_noext = 'srcd_{}_HirschPt2011'.format(rawname_)

rawname_LFPonly = rawname_ + '_LFPonly'+ '.fif'
rawname_LFPonly_full = os.path.join( data_dir, rawname_LFPonly )
raw_lfponly = mne.io.read_raw_fif(rawname_LFPonly_full, None)
# if use_lfp_HFO:
#     raw_lfp_highres = upre.saveLFP_nonresampled(rawname_, skip_if_exist = 1)
#     if raw_lfp_highres is None:
#         lfp_fname_full = os.path.join(data_dir, '{}_LFP_1kHz.fif'.format(rawname_) )
#         raw_lfp_highres = mne.io.read_raw_fif(lfp_fname_full)

#     sfreq_highres = raw_lfp_highres.info['sfreq']
#     sfreq_highres = int(sfreq_highres)

newsrc_fname_full = os.path.join( data_dir, 'av_' + src_fname_noext + '.fif' )
raw_srconly =  mne.io.read_raw_fif(newsrc_fname_full, None)




In [ ]:
raw_lfponly.load_data()
raw_srconly.load_data()

In [ ]:
fbands = {'tremor': [3,10], 'low_beta':[11,22], 'high_beta':[22,30],
           'low_gamma':[30,60], 'high_gamma':[60,90],
          'HFO1':[91,200], 'HFO2':[200,300], 'HFO3':[300,400],
          'beta':[15,30],   'gamma':[30,100], 'HFO':[91,400]}



In [ ]:
gc.collect()

In [ ]:
bandPairs = [('tremor','tremor', 'corr'),
             ('low_beta','low_beta', 'corr'), ('high_beta','high_beta', 'corr'),
             ('low_gamma','low_gamma', 'corr') , ('high_gamma','high_gamma', 'corr') ]

for bp in bandPairs:
#     fbname0 = 'tremor'
#     fbname1 = 'low_beta'
    fbname0,fbname1,_ = bp
    fband0 = fbands[fbname0]
    fband1 = fbands[fbname1]

    lfp_bp = raw_lfponly.copy()
    lfp_bp.filter(l_freq=fband0[0],h_freq=fband0[1])

    src_bp = raw_srconly.copy()
    src_bp.filter(l_freq=fband1[0], h_freq=fband1[1])
    
    plotCorrelationsAllChannelParis(lfp_bp,src_bp,fbname0,fbname1)

In [ ]:
# from scipy.ndimage.interpolation import shift
# shift(np.arange(10), 4, mode='nearest')

In [ ]:
def getNormData(dat0, q = 0.05):
    assert q >= 0
    if q < 1e-10:
        low0 = np.min(dat0)
        high0 = np.max(dat0)
        me0 = np.mean(dat0)
    else:
        low0,high0 = np.percentile(dat0, [100*q,100*(1-q)] )
        me0 = np.mean( dat0 [(dat0 >= low0) * (dat0 <= high0) ]  )
    return me0, low0, high0

In [ ]:
sfreq = 256

In [ ]:
sfreq = int(raw_lfponly.info['sfreq'] )


In [ ]:
# td1 = np.random.uniform(size=40*1000)
# td2 = np.random.uniform(size=40*1000)
# chdata0 = td1
# chdata1 = td2

In [ ]:
def getCorrelFun(dat0,dat1, timebin, windowsz, lags, normdata=None):
#     assert timebin - min(lags) > 0
#     assert timebin + max(lags) < len(dat0)
    #assert np.max(np.abs(lags)) < windowsz//2
    res = []
    if normdata is None:
        normdata = {}
        me0, low0 ,high0 = getNormData(dat0)
        me1, low1 ,high1 = getNormData(dat1)
        normdata['me0'] = me0
        normdata['me1'] = me1
        normdata['range0'] = (high0-low0)
        normdata['range1'] = (high1-low1)
    for nbins_lag in lags:
        if (timebin + nbins_lag - windowsz//2 < 0) or (timebin + nbins_lag + windowsz//2 >= len(dat0 ) ):
            r = np.nan
            res += [r]
            continue 
        #print(timebin - windowsz//2, timebin + windowsz//2)
        subdat0 = dat0[timebin - windowsz//2: timebin + windowsz//2] 
        subdat1 = dat1[timebin + nbins_lag - windowsz//2: timebin + nbins_lag + windowsz//2] 
        
        if isinstance(normdata, str) and normdata == 'loc':
            normdata_loc = {}
            me0, low0 ,high0 = getNormData(subdat0)
            me1, low1 ,high1 = getNormData(subdat1)
            normdata_loc['me0'] = me0
            normdata_loc['me1'] = me1
            normdata_loc['range0'] = (high0-low0)
            normdata_loc['range1'] = (high1-low1)
            
            normdata_cur = normdata_loc
        else:
            normdata_cur = normdata
            
        #subdat1 - normdata_cur['me1']  -- this would spoil oroignal data
        
        #shifted = shift(dat1, nbins_lag, mode='nearest')
        r = np.correlate( subdat0 - normdata_cur['me0'], subdat1 - normdata_cur['me1'] ) / windowsz
        #r = subdat0 * subdat1 
        norm_mult = ( normdata_cur['range0'] *normdata_cur['range1'] )
        #print(norm_mult)
        r /= norm_mult
        
        res += [r]
        
    return np.array(res)

#r = getCorrelFun(chdata0,chdata1, 1000, windowsz, lags, normdata=normdata)
#r

In [ ]:
def plotCorrelationsAllChannelParis(lfp_bp,src_bp, fbname0,fbname1):
    chis = mne.pick_channels_regexp(src_bp.ch_names, 'msrcR*_allf.*')
    chns_src = [src_bp.ch_names[ind] for ind in chis]
    chis = mne.pick_channels_regexp(lfp_bp.ch_names, 'LFPR.*')
    chns_lfp = [lfp_bp.ch_names[ind] for ind in chis]
    chns_all = chns_src + chns_lfp

    chn_pairs = []
    for chn0 in chns_all:
        for chn1 in chns_all:
            if chn0.find('LFP') >=0  and chn1.find('LFP') >= 0:
                continue
            chn_pairs += [(chn0,chn1)]

    anns_fn = rawname_ + '_anns.txt'
    anns_fn_full = os.path.join(data_dir, anns_fn)
    anns = mne.read_annotations(anns_fn_full)

    lims = (-0.1,0.1)
    
    diam = 2*sfreq
    step = sfreq / 8
    step = sfreq / 16
    lags = np.arange(-diam,diam,step, dtype=int)
    windowsz = sfreq * 4

    #tbins = np.arange(0,300,windowsz/sfreq, dtype=int)*sfreq
    tbins = np.arange(0,raw_lfponly.times[-1]-np.max(lags)/sfreq,windowsz/sfreq, dtype=int)*sfreq


    N = len(chn_pairs)

    nr = N; nc=1
    hh = 3; ww = 20
    fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh), sharex='col')
    plt.subplots_adjust(bottom=0.01,top =0.98, left=0.04, right=0.99)

    for i in range(nr):
        ax = axs[i]; 
        p = chn_pairs[i]
        chn0, chn1 = p

        plotCrossCorr(ax,lfp_bp, src_bp, chn0,chn1, tbins, lags, windowsz, lims = lims)

        for di in range(len(anns)):
            ann_cur = anns[di]
            if ann_cur['description'] in ['trem_L', 'hold_L', 'notrem_L']:
                ax.axvline(ann_cur['onset'], ls=':', c='r')
                ax.axvline(ann_cur['onset']+ann_cur['duration'], ls=':', c='r')
            ax.set_xlim(tbins[0]/sfreq,tbins[-1]/sfreq)


    figtitle = '{} vs {}'.format(fbname0,fbname1)
    plt.suptitle(figtitle, size=28)
    plt.savefig('{}__{}_cross_correl.png'.format(rawname_,figtitle))

In [ ]:
def plotCrossCorr(ax, lfp_bp, src_bp, chn0,chn1, tbins, lags, windowsz, 
                  q=2e-2, lims=(-0.15,0.15), printLog=0, cmap='bwr'):
    #chn1 = 'msrcR_allf_2'
    #chn1 = 'msrcR_allf_0'
    chns = [chn0 ,chn1]

    chdatas = []
    for chn in chns:
        if chn.find('LFP') >= 0:
        #src_chname = 'msrcR_allf_0'
        #lfp_chname = 'LFPR01'
            chdata,times = lfp_bp[chn]
        else:
            chdata,times = src_bp[chn]
        chdatas += [chdata[0]]

    chdata0, chdata1 = chdatas


    normdata = {}
    me0, low0 ,high0 = getNormData(chdata0, q =q)
    me1, low1 ,high1 = getNormData(chdata1, q=q)
    normdata['me0'] = me0
    normdata['me1'] = me1
    normdata['range0'] = (high0-low0)
    normdata['range1'] = (high1-low1)

    if printLog:
        print(normdata)
    
    rs = []
    for i,timebin in enumerate(tbins):
        #r = getCorrelFun(chdata0,chdata1, timebin, windowsz, lags, normdata='loc')
        r = getCorrelFun(chdata0,chdata1, timebin, windowsz, lags, normdata=normdata)
        #r = getCorrelFun(td1,td2, timebin, windowsz, lags, normdata='loc')

        rs += [r.T]
    rs = np.vstack(rs)

    import matplotlib.colors as mcolors
    norm = mcolors.Normalize(vmin=lims[0], vmax = lims[1])


    ax.pcolor(tbins/sfreq,lags/sfreq, rs.T, norm=norm, cmap=cmap)
    ax.set_ylabel('lag, [s]')
    ax.set_xlabel('time, [s]')
    ax.set_title('{} vs {}'.format(chns[0],chns[1]))
    
    return tbins,lags,rs

In [ ]:
chn0 = 'msrcR_allf_0'
chn1 = 'LFPR01'
plt.figure(figsize=(17,5))
ax = plt.gca()
plotCrossCorr(ax,chn0,chn1)

In [ ]:
chn0 = 'msrcR_allf_0'
chn1 = 'msrcR_allf_2'
plt.figure(figsize=(17,5))
ax = plt.gca()
plotCrossCorr(ax,chn0,chn1)

In [ ]:
lfp_bp.ch_names

In [ ]:
chdata0 = lfp_bp['LFPR12'][0][0]
chdata1 = lfp_bp['LFPR23'][0][0]

In [ ]:
diam = 2*sfreq
step = sfreq / 8
step = sfreq / 16
lags = np.arange(-diam,diam,step, dtype=int)
windowsz = sfreq * 4

In [ ]:
tbins = np.arange(20,30,2, dtype=int)*sfreq
#%debug
nr = len(tbins); nc=1
hh = 3; ww = 10
fig,axs = plt.subplots(nr,nc,figsize=(nc*ww,nr*hh))

ylims=(-0.15,0.15)
rs = []
for i,timebin in enumerate(tbins):
    #r = getCorrelFun(chdata0,chdata1, timebin, windowsz, lags, normdata='loc')
    r = getCorrelFun(chdata0,chdata1, timebin, windowsz, lags, normdata=None)
    #r = getCorrelFun(td1,td2, timebin, windowsz, lags, normdata='loc')
    ax = axs[i]
    ax.plot(lags/sfreq,r)
    ax.axhline(0,ls=':')
    ax.set_title('time = {:.1f}'.format(timebin/sfreq)  )
    ax.set_ylim(ylims)

In [ ]:
r

In [ ]:
import gc; gc.collect()

In [ ]:
plt.close('all')